In [ ]:
import numpy as np
import pandas as pd
import os
import plotly.express as px
import scipy
import skimage

folder=r"J:\ctgroup\Edward\DATA\VMI\20250303\Propylene Oxide 2W\calibrated"

dfs={}

for file in os.listdir(folder):
    if file.endswith(".h5"):
        angle=int(file.split('_')[0])
        print(file)
        dfs[angle]=pd.read_hdf(os.path.join(folder,file))

In [ ]:
angle=0
df=dfs[angle]
display(df.head())
df=df[df['m/q']>0]
df=df[df['m/q']<70]
fig=px.histogram(df, x='m/q',nbins=1000,title=f'Propylene Oxide Ion Spectrum',range_x=[0,70], height=800, width=1600
                 ).show()

In [ ]:
peaks=15.25,27.75,44.75,59
widths=2,5,5,5
full_dataframe=pd.DataFrame()
for i, (peak, width) in enumerate(zip(peaks,widths)):
    print(i,peak,width)
    for angle, df in dfs.items():
        df=df[df['m/q']>peak-width]
        df=df[df['m/q']<peak+width]
        df['angle']=angle
        df['ellipticity']=np.tan(np.radians(angle))
        df['fragment_id']=i
        full_dataframe=pd.concat([full_dataframe,df])

In [ ]:
full_dataframe["pr"]=np.sqrt(full_dataframe['px']**2+full_dataframe['py']**2+full_dataframe['pz']**2)
full_dataframe["theta"]=np.arccos(full_dataframe['py']/full_dataframe['pr'])
full_dataframe["phi"]=np.arctan2(full_dataframe['px'],full_dataframe['pz'])-np.radians(full_dataframe['angle'])
full_dataframe.sort_values(by='angle',inplace=True)

display(full_dataframe.head())

In [ ]:
peak_idxs=[]
for angle in full_dataframe['angle'].unique():
    df=full_dataframe[full_dataframe['angle']==angle]
    hist,re=np.histogram(df['pr'],bins=100,range=(0,0.8),density=True)
    re=(re[1:]+re[:-1])/2
    signal=skimage.filters.difference_of_gaussians(hist,low_sigma=2,high_sigma=5)
    peaks, _=scipy.signal.find_peaks(np.maximum(signal,0))
    peaks=peaks[re[peaks]>0.1]
    peak_widths=scipy.signal.peak_widths(signal,peaks)[0]
    peak_widths=scipy.interpolate.interp1d(range(len(re)),re)(peak_widths)
    peak_dists=[abs((df['pr']-re[peak])/width)for peak,width in zip(peaks,peak_widths)]
    peak_dists=np.array(peak_dists)
    peak_idx=peak_dists.argmin(axis=0)
    peak_idx=np.where(peak_dists.min(axis=0)<1,peak_idx,-1)
    peak_idxs+=peak_idx.tolist()
    print(angle,re[peaks])
    # fig=px.line(x=re,y=hist)
    # fig.add_scatter(x=re[peaks],y=hist[peaks], mode='markers', error_x=dict(type='data',array=peak_widths))
    # fig.show()
    # 

full_dataframe['peak_idx']=peak_idxs
display(full_dataframe.head())

In [ ]:
def get_real_beta(l,m,theta,phi):
    if m==0:
        return np.real(scipy.special.sph_harm(m,l,phi,theta))
    if m>0:
        return (-1)**m*np.sqrt(2)*np.real(scipy.special.sph_harm(m,l,phi,theta))
    if m<0:
        return (-1)**m*np.sqrt(2)*np.imag(scipy.special.sph_harm(-m,l,phi,theta))
    
lmax=6
for l in range(lmax+1):
    for m in range(-l,l+1):
        full_dataframe[f'beta_{l}_{m}']=get_real_beta(l,m,full_dataframe['theta'],full_dataframe['phi'])
        
full_dataframe['fba']=full_dataframe['py']/np.abs(full_dataframe['py'])


In [ ]:
grouped_df=full_dataframe.groupby(['angle','fragment_id','peak_idx']).mean()
grouped_uncertainty=full_dataframe.groupby(['angle','fragment_id','peak_idx']).sem()
grouped_uncertainty.columns=[f'{col}_uncertainty' for col in grouped_uncertainty.columns]
grouped_df= pd.concat([grouped_df,grouped_uncertainty],axis=1)
grouped_df.reset_index(inplace=True)

# Create a list to store all rows
rows = []

# Get unique values once to avoid repeated calls
angles = full_dataframe['angle'].unique()
fragment_ids = full_dataframe['fragment_id'].unique()
peak_idxs = full_dataframe['peak_idx'].unique()

# Loop through each combination
for angle in angles:
    for fragment_id in fragment_ids:
        for peak_idx in peak_idxs:
            # Filter dataframe once for this combination
            filtered = grouped_df[(grouped_df['angle'] == angle) &
                                 (grouped_df['fragment_id'] == fragment_id) &
                                 (grouped_df['peak_idx'] == peak_idx)]

            if filtered.empty:
                continue

            # For each l, m pair in this filtered result
            for l in range(lmax + 1):
                for m in range(-l, l + 1):
                    # Create a dictionary for this row

                    es= l%2==1
                    dichroic= (l%2==1 and m>=0) or (l%2==0 and m<0)
                    ty=""
                    ty+= "Dichroic" if dichroic else ""
                    ty+= " and " if dichroic and es else ""
                    ty+= "Enantiosensitive" if es else ""
                    if not ty:
                        ty="Neither"

                    rows.append({
                        'angle': angle,
                        'fragment_id': fragment_id,
                        'peak_idx': peak_idx,
                        'l': l,
                        'm': m,
                        'value': filtered[f'beta_{l}_{m}'].values[0],
                        'uncertainty': filtered[f'beta_{l}_{m}_uncertainty'].values[0],
                        'allowed': m%2==0,
                        'enantiosensitive': es,
                        'dichroic': dichroic,
                        'type': ty
                    })


# Create DataFrame from the list of dictionaries in one operation
beta_df = pd.DataFrame(rows)
display(beta_df.head())
px.line(grouped_df[grouped_df["peak_idx"]==0], x='angle',y='fba', color='fragment_id',error_y='fba_uncertainty',title='FBA vs Angle',height=800, width=1600).show()

diff_map = {}
delta_diff_map = {}
for fragment_id in np.unique(grouped_df['fragment_id']):
    for angle in np.unique(grouped_df['angle']):
        for peak_idx in np.unique(grouped_df['peak_idx']):
            if grouped_df[
                (grouped_df['fragment_id'] == fragment_id) &
                (grouped_df['angle'] == angle) &
                (grouped_df['peak_idx'] == peak_idx)
            ].empty:
                continue
            
            diff_map[(fragment_id, angle, peak_idx)] = grouped_df[
                (grouped_df['fragment_id'] == fragment_id) &
                (grouped_df['angle'] == angle) &
                (grouped_df['peak_idx'] == peak_idx)
            ]['fba'].values[0] - grouped_df[
                (grouped_df['fragment_id'] == fragment_id) &
                (grouped_df['angle'] == -angle) &
                (grouped_df['peak_idx'] == peak_idx)
            ]['fba'].values[0]
            
            delta_diff_map[(fragment_id, angle, peak_idx)] = (grouped_df[
                (grouped_df['fragment_id'] == fragment_id) &
                (grouped_df['angle'] == angle) &
                (grouped_df['peak_idx'] == peak_idx)
            ]['fba_uncertainty'].values[0]**2+ grouped_df[
                (grouped_df['fragment_id'] == fragment_id) &
                (grouped_df['angle'] == -angle) &
                (grouped_df['peak_idx'] == peak_idx)
            ]['fba_uncertainty'].values[0]**2)**0.5
            

# Create a new column in the beta_df DataFrame to store the fba difference
grouped_df['fba_diff'] = grouped_df.apply(lambda row: diff_map[(row['fragment_id'], row['angle'], row['peak_idx'])], axis=1)
grouped_df['fba_diff_uncertainty'] = grouped_df.apply(lambda row: delta_diff_map[(row['fragment_id'], row['angle'], row['peak_idx'])], axis=1)

px.line(grouped_df[grouped_df["peak_idx"]==0], x='angle',y='fba_diff', color='fragment_id',error_y='fba_diff_uncertainty',title='FBA vs Angle',height=800, width=1600).show()


# Display the result
display(grouped_df[['angle', 'fragment_id', 'peak_idx', 'fba', 'fba_diff']].head())

In [ ]:
filtered=beta_df[(beta_df['peak_idx']==0) & (beta_df['l']<8) &(beta_df['type']=='Dichroic and Enantiosensitive')&(beta_df['allowed']) & (beta_df['fragment_id']==3)]
filtered['label']=filtered['l'].astype(str)+","+filtered['m'].astype(str)
fig=px.line(filtered, x='angle',y='value',error_y='uncertainty',line_group='label', facet_col_wrap=2, height=800, width=1600, symbol='m', title='Beta Coefficients', color='l')

fig.show()

beta_diff_map = {}
delta_beta_diff_map = {}

for l in np.unique(beta_df['l']):
    for m in np.unique(beta_df['m']):
        for fragment_id in np.unique(beta_df['fragment_id']):
            for angle in np.unique(beta_df['angle']):
                for peak_idx in np.unique(beta_df['peak_idx']):
                    if beta_df[
                        (beta_df['l'] == l) &
                        (beta_df['m'] == m) &
                        (beta_df['fragment_id'] == fragment_id) &
                        (beta_df['angle'] == angle) &
                        (beta_df['peak_idx'] == peak_idx)
                    ].empty:
                        continue

                    beta_diff_map[(l, m, fragment_id, angle, peak_idx)] = beta_df[
                        (beta_df['l'] == l) &
                        (beta_df['m'] == m) &
                        (beta_df['fragment_id'] == fragment_id) &
                        (beta_df['angle'] == angle) &
                        (beta_df['peak_idx'] == peak_idx)
                    ]['value'].values[0] - beta_df[
                        (beta_df['l'] == l) &
                        (beta_df['m'] == m) &
                        (beta_df['fragment_id'] == fragment_id) &
                        (beta_df['angle'] == -angle) &
                        (beta_df['peak_idx'] == peak_idx)
                    ]['value'].values[0]

                    delta_beta_diff_map[(l, m, fragment_id, angle, peak_idx)] = (beta_df[
                        (beta_df['l'] == l) &
                        (beta_df['m'] == m) &
                        (beta_df['fragment_id'] == fragment_id) &
                        (beta_df['angle'] == angle) &
                        (beta_df['peak_idx'] == peak_idx)
                    ]['uncertainty'].values[0]**2 + beta_df[
                        (beta_df['l'] == l) &
                        (beta_df['m'] == m) &
                        (beta_df['fragment_id'] == fragment_id) &
                        (beta_df['angle'] == -angle) &
                        (beta_df['peak_idx'] == peak_idx)
                    ]['uncertainty'].values[0]**2)**0.5
beta_df['beta_diff'] = beta_df.apply(lambda row: beta_diff_map[(row['l'], row['m'], row['fragment_id'], row['angle'], row['peak_idx'])], axis=1)
beta_df['beta_diff_uncertainty'] = beta_df.apply(lambda row: delta_beta_diff_map[(row['l'], row['m'], row['fragment_id'], row['angle'], row['peak_idx'])], axis=1)


In [ ]:
filtered=beta_df[(beta_df['peak_idx']==0) & (beta_df['l']<8)&(beta_df['allowed']) & (beta_df['fragment_id']==3)]
filtered['label']=filtered['l'].astype(str)+","+filtered['m'].astype(str)
fig=px.line(filtered, x='angle',y='beta_diff',error_y='beta_diff_uncertainty',line_group='label', facet_col_wrap=2, height=800, width=1600, symbol='m', title='Beta Coefficients', color='l', facet_col='type')
fig.show()